In [1]:
import pandas as pd
from pathlib import Path

base_dir = Path(r"I:\NTS\classified builds")
output_dir = Path(r"I:\NTS\outputs\cwac tables")

In [2]:
v12 = pd.read_csv(base_dir / "cb_tfn_v12.csv")

C:\Users\Predator\AppData\Local\Temp\ipykernel_11580\539251893.py:1: DtypeWarning: Columns (10,18,39,50) have mixed types. Specify dtype option on import or set low_memory=False.
  v12 = pd.read_csv(base_dir / "cb_tfn_v12.csv")


In [13]:
v12.columns

Index(['PSUID', 'PSUPopDensity', 'PSUAreaType1_B01ID', 'PSUAreaType2_B01ID',
       'PSUGOR_B02ID', 'PSUCounty_B01ID', 'SurveyYear', 'HouseholdID',
       'ONSACSub_B01ID', 'SettlementEW_B01ID', 'HHoldOSWard_B01ID',
       'HHoldOSLAUA_B01ID', 'HHoldCounty_B01ID', 'HHoldGOR_B02ID',
       'HHoldAreaType1_B01ID', 'HHoldAreaType2_B01ID', 'HHoldNumAdults',
       'NumCarVan_B02ID', 'NumCarVan', 'W1', 'W2', 'W3', 'IndividualID',
       'Age_B01ID', 'Sex_B01ID', 'XSOC2000_B02ID', 'NSSec_B03ID',
       'SIC2007_B02ID', 'CarAccess_B01ID', 'DrivLic_B02ID', 'EcoStat_B01ID',
       'EcoStat_B02ID', 'DayID', 'TravDay', 'TravelWeekDay_B01ID',
       'TravelWeekDay_B02ID', 'TripID', 'JJXSC', 'JOTXSC', 'JTTXSC', 'JD',
       'MainMode_B03ID', 'MainMode_B04ID', 'MainMode_B11ID',
       'TripPurpFrom_B01ID', 'TripPurpTo_B01ID', 'TripStart_B01ID',
       'TripEnd_B01ID', 'TripDisIncSW', 'TripDisIncSW_B01ID', 'TripTravTime',
       'TripOrigCounty_B01ID', 'TripDestCounty_B01ID', 'TripDestUA2009_B01ID',


In [3]:
edges = [0, 10, 25, 50, 100, 200, 1000]
labels = ['0-10', '10-25','25-50','50-100','100-200','200+']
v12['dist_bins'] = pd.cut(v12.TripDisIncSW, bins=edges, labels=labels)

In [4]:
#lookups
modes = {1: 'Walk',
  2: 'Cycle',
  3: 'Car',
  4: 'Van',
  5: 'Bus',
  6: 'Rail',
  7: 'Light Rail',
  8: 'Air'}

purposes = {1: 'Commute',
  2: 'Business',
  3: 'Education',
  4: 'Shopping',
  5: 'PB',
  6: 'Social',
  7: 'Visit Friends',
  8: 'Holiday - Day trip',
  11: 'Escort commute',
  12: 'Escort business',
  13: 'Education',
  14: 'Escort shopping'}

area_dict = {1: 'Inner London',
  2: 'Outer London built-up areas',
  3: 'West Midlands built-up areas',
  4: 'Greater Manchester built-up areas',
  5: 'West Yorkshire built-up areas',
  6: 'Glasgow built-up areas',
  7: 'Liverpool built-up areas',
  8: 'Tyneside built-up areas',
  9: 'South Yorkshire built-up areas',
  10: 'Other urban area - over 250k population',
  11: 'Other urban area - 100k to 250k population',
  12: 'Other urban area - 50k to 100k population',
  13: 'Other urban area - 25k to 50k population',
  14: 'Other urban area - 10k to 25k population',
  15: 'Other urban area - 3k to 10k population',
  16: 'Rural',
  -8: 'nan',
  -9: 'DNA'}

other_dic = {0.0: 'other',
 'Shopping': 'other',
 'Social': 'other',
 'Visit Friends': 'other',
 'Holiday - Day trip': 'other',
 'Escort commute': 'other',
 'Escort business': 'other',
 'Escort shopping': 'other'}

In [6]:
v12['main_mode'].replace(modes, inplace=True)
v12['trip_purpose'].replace(purposes, inplace=True)
v12['trip_purpose'].replace(other_dic, inplace=True)
v12['HHoldAreaType2_B01ID'].replace(area_dict, inplace=True)

In [7]:
cwac = v12[(v12['TripOrigUA2009_B01ID'] == 154)|(v12['TripDestUA2009_B01ID'] == 154)]
cwac.loc[cwac['TripOrigUA2009_B01ID'] != 154, 'TripOrigUA2009_B01ID'] = 'External'
cwac.loc[cwac['TripDestUA2009_B01ID'] != 154, 'TripDestUA2009_B01ID'] = 'External'
cwac.loc[cwac['TripDestUA2009_B01ID'] == 154, 'TripDestUA2009_B01ID'] = 'Internal'
cwac.loc[cwac['TripOrigUA2009_B01ID'] == 154, 'TripOrigUA2009_B01ID'] = 'Internal'
cwac.rename(columns = {'TripOrigUA2009_B01ID':"Orig", 'TripDestUA2009_B01ID':"Dest"}, inplace=True)

C:\Users\Predator\AppData\Local\Temp\ipykernel_11580\513449330.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cwac.rename(columns = {'TripOrigUA2009_B01ID':"Orig", 'TripDestUA2009_B01ID':"Dest"}, inplace=True)


In [8]:
cwac.loc[cwac['HHoldAreaType2_B01ID'] != 'Rural', 'HHoldAreaType2_B01ID'] = 'Urban'

In [9]:
cwac.trip_purpose.fillna('other', inplace=True)

C:\Users\Predator\AppData\Local\Temp\ipykernel_11580\3322683356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cwac.trip_purpose.fillna('other', inplace=True)


In [12]:
occupancy = cwac[cwac['main_mode'].str.contains('Car', na=False)].groupby(['HHoldAreaType2_B01ID','trip_purpose','dist_bins','occupant'])['weighted_trips'].sum().unstack()

In [14]:
occupancy['occupancy'] = (occupancy['driver'] + occupancy['passenger']) / occupancy['passenger']

In [17]:
occupancy.driver.sum() + occupancy.passenger.sum()

26168.228646322976

In [22]:
cwac.weighted_trips.sum()

35502.35184819163

In [23]:
occupancy.to_csv(output_dir / 'occupapncy.csv')

In [21]:
cwac.groupby(['Orig','Dest', 'main_mode', 'trip_purpose', 'dist_bins', 'HHoldAreaType2_B01ID']).sum()['weighted_trips']#.to_csv(output_dir / 'pivotable.csv')

C:\Users\Predator\AppData\Local\Temp\ipykernel_10516\302994789.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cwac.groupby(['Orig','Dest', 'main_mode', 'trip_purpose', 'dist_bins', 'HHoldAreaType2_B01ID']).sum()['weighted_trips']#.to_csv(output_dir / 'pivotable.csv')


Orig      Dest      main_mode  trip_purpose  dist_bins  HHoldAreaType2_B01ID
External  External  Air        Business      0-10       Rural                   0.0
                                                        Urban                   0.0
                                             10-25      Rural                   0.0
                                                        Urban                   0.0
                                             25-50      Rural                   0.0
                                                                               ... 
Internal  Internal  Walk       other         50-100     Urban                   0.0
                                             100-200    Rural                   0.0
                                                        Urban                   0.0
                                             200+       Rural                   0.0
                                                        Urban                   0.0

In [37]:
#average mode share
cwac.groupby(['SurveyYear', 'Orig', 'Dest', 'main_mode'])['weighted_trips'].sum().to_csv(output_dir / 'average_mode_share.csv')

In [31]:
#modes by purpose
cwac.groupby(['SurveyYear', 'Orig','Dest', 'trip_purpose', 'main_mode'])['weighted_trips'].sum().to_csv(output_dir / "average_mode_by_purpose.csv")

In [32]:
#mode by distance
cwac.groupby(['SurveyYear', 'Orig','Dest', 'dist_bins', 'main_mode'])['weighted_trips'].sum().to_csv(output_dir / "average_mode_by_distance.csv")

In [33]:
cwac.groupby(['SurveyYear', 'Orig','Dest','HHoldAreaType2_B01ID', 'main_mode'])['weighted_trips'].sum().to_csv(output_dir / "average_mode_by_RUC.csv")

In [34]:
cwac.groupby(['Orig','Dest','dist_bins'])['weighted_trips'].sum().to_csv(output_dir / "TLD.csv")

In [37]:
cwac.weighted_trips.sum()

35502.35184819163

In [3]:
import asyncio
from datetime import datetime

from traveltimepy import Location, Coordinates, PublicTransport, Property, FullRange, TravelTimeSdk

async def main():
    sdk = TravelTimeSdk("YOUR_APP_ID", "YOUR_APP_KEY")

    locations = [
        Location(id="London center", coords=Coordinates(lat=51.508930, lng=-0.131387)),
        Location(id="Hyde Park", coords=Coordinates(lat=51.508824, lng=-0.167093)),
        Location(id="ZSL London Zoo", coords=Coordinates(lat=51.536067, lng=-0.153596))
    ]

    results = await sdk.time_filter_async(
        locations=locations,
        search_ids={
            "London center": ["Hyde Park", "ZSL London Zoo"],
            "ZSL London Zoo": ["Hyde Park", "London center"],
        },
        departure_time=datetime.now(),
        travel_time=3600,
        transportation=PublicTransport(type="bus"),
        properties=[Property.TRAVEL_TIME],
        range=FullRange(enabled=True, max_results=3, width=600)
    )

    print(results)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

RuntimeError: asyncio.run() cannot be called from a running event loop